In [3]:
# Import the function that creates all database tables
from database import create_tables

# Execute the function to create the tables defined by the SQLAlchemy models
create_tables()

2025-11-21 12:11:56,530 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-21 12:11:56,531 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("pipe")
2025-11-21 12:11:56,531 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-21 12:11:56,532 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("weather_station")
2025-11-21 12:11:56,532 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-21 12:11:56,533 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("rainfall")
2025-11-21 12:11:56,534 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-21 12:11:56,534 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("air_humidity")
2025-11-21 12:11:56,535 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-21 12:11:56,535 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("air_temperature")
2025-11-21 12:11:56,536 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-21 12:11:56,536 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("pipe_seismic_impact")
2025-11-21 12:11:56,536 INFO sqlalc

In [4]:
# Import the session factory configured in the database module
from database import SessionLocal

# Create a new SQLAlchemy session to interact with the database
session = SessionLocal()

In [5]:
import pandas as pd
from sqlalchemy.orm import Session

# ---------------------------------------------------------------------------
# 1. Read Excel file into a DataFrame
# ---------------------------------------------------------------------------

def read_excel_to_dataframe(
    excel_path: str,
    sheet_name: int | str = 0,
    drop_empty_rows: bool = True,
) -> pd.DataFrame:
    """
    Read an Excel file into a pandas DataFrame.

    Parameters
    ----------
    excel_path : str
        Path to the Excel file on disk.
    sheet_name : int or str, optional
        Sheet index (0-based) or sheet name. Default is 0 (first sheet).
    drop_empty_rows : bool, optional
        If True, rows that are completely empty will be removed.

    Returns
    -------
    df : pandas.DataFrame
        DataFrame containing the data from the selected sheet.
    """
    df = pd.read_excel(excel_path, sheet_name=sheet_name)

    if drop_empty_rows:
        df = df.dropna(how="all")

    return df

In [6]:
# ---------------------------------------------------------------------------
# 2. Upload a DataFrame to a database table (generic)
# ---------------------------------------------------------------------------

def upload_dataframe_to_table(
    df: pd.DataFrame,
    model_cls,
    engine,
):
    """
    Upload a pandas DataFrame to a database table using SQLAlchemy.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the data to be inserted.
        Column names should match the attributes of `model_cls`.
    model_cls : Base subclass
        SQLAlchemy ORM model class corresponding to the target table.
    engine : sqlalchemy.Engine
        SQLAlchemy engine connected to the target database.

    Notes
    -----
    - This function:
        * keeps only the columns that exist in the model's table,
        * replaces pandas NA values with Python None,
        * uses bulk_insert_mappings for efficient insertion.
    - It assumes the table is already created in the database.
    """

    # 1. Keep only columns that exist in the model
    model_columns = {col.name for col in model_cls.__table__.columns}
    valid_cols = [c for c in df.columns if c in model_columns]
    df_clean = df[valid_cols].copy()

    # 2. Replace pandas NA with None so SQLAlchemy/DB can handle null values
    records = (
        df_clean
        .where(pd.notna(df_clean), None)
        .to_dict(orient="records")
    )

    # 3. Bulk insert into the database
    with Session(engine) as session:
        session.bulk_insert_mappings(model_cls, records)
        session.commit()


In [7]:
# ---------------------------------------------------------------------------
# 3. Example usage (to be adapted in your own scripts / notebooks)
# ---------------------------------------------------------------------------

if __name__ == "__main__":
    # Example: import the engine and models from other modules
    # from database import engine
    # from schema import (
    #     Pipe,
    #     Manhole,
    #     HydraulicProperties,
    #     Inspection,
    #     Defect,
    #     WeatherStation,
    #     Rainfall,
    # )

    # Example 1: load pipes
    # pipes_df = read_excel_to_dataframe("documents/PIPES.xlsx", sheet_name=0)
    # upload_dataframe_to_table(pipes_df, Pipe, engine)

    # Example 2: load manholes
    # manholes_df = read_excel_to_dataframe("documents/MANHOLES.xlsx", sheet_name=0)
    # upload_dataframe_to_table(manholes_df, Manhole, engine)

    pass